In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Setting up the Vacant Building DataSet

In [69]:
#publicBuilding = pd.read_csv('Public_Buildings_-_Dataset.csv')

VacantBuilding = pd.read_csv('Vacant_Buildings_-_Dataset.csv')
print(VacantBuilding.head())

#Set default column and values
#VacantBuilding['Latitude'] = 43.806
#VacantBuilding['Longitude'] = -79.194


VacantBuilding.loc[VacantBuilding['WARD'] == 1.0]
VacantBuilding.iloc[0:,7]

#print(VacantBuilding.iloc[1,7])

a= VacantBuilding['MAP LOCATION'].str.find(',')


# new data frame with split value columns 
new=VacantBuilding['MAP LOCATION'].str.split(",", n = 1, expand = True) 

#print(new[0])
    
# making seperate first name column from new data frame 
VacantBuilding['Latitude']= pd.to_numeric(new[0].str[1:]) #pd.to_numeric convert it to float

# making seperate last name column from new data frame 
VacantBuilding['Longitude']= pd.to_numeric(new[1].str.rstrip(')'))
#VacantBuilding['Longitude']= 
#print(VacantBuilding['Longitude'].str.rstrip(')'))
#VacantBuilding["Name"].str.find(sub) 

#VacantBuilding['date1'] = pd.to_datetime(VacantBuilding['VACANT AS OF'], format='%m/%d/%Y', utc=True)


VacantBuilding['Date']= pd.to_datetime(VacantBuilding['VACANT AS OF'])


print(VacantBuilding.head())
print(VacantBuilding.dtypes)

         ADDRESS            VACANT AS OF              DWELLING TYPE  \
0   663 3RD ST E  04/26/2011 12:00:00 AM   Multi-family Residential   
1   989 3RD ST E  11/01/2011 12:00:00 AM  Single Family Residential   
2  1457 3RD ST E  10/01/2014 12:00:00 AM  Single Family Residential   
3   249 4TH ST E  10/13/2016 12:00:00 AM                 Commercial   
4   767 4TH ST E  01/09/2009 12:00:00 AM                     Duplex   

   VACANT BUILDING CATEGORY  WARD  DISTRICT  CENSUS TRACT  \
0                         2   7.0       4.0       34400.0   
1                         2   7.0       4.0       34500.0   
2                         2   7.0       1.0       34602.0   
3                         2   2.0      17.0       34201.0   
4                         2   7.0       4.0       34500.0   

                           MAP LOCATION  
0  (44.9547963325615, -93.071254461297)  
1  (44.9568544056193, -93.058822503618)  
2  (44.9569709429740, -93.038835961878)  
3  (44.9487456956542, -93.085933397554

In [137]:
#Add  demographic Data
SPDem = pd.read_csv('MSP Neighborhoods_2013-2017.csv', skiprows=[1])
SPDem.columns = SPDem.columns.str.replace(' ', '')
SPDem['City'].value_counts().to_frame()
SPDem= SPDem.query('geography in ["Summit Hill","Hamline-Midway","Frogtown/Thomas-Dale","Union Park"]')

#SPDem.columns.values[i]

#for i,square in enumerate(SPDem.columns.values): 
#    print(str(i) + ' '+ square)
#print(SPDem)

SPDem = SPDem.iloc[:,[1,2,27,39,43,47,63,71,107,123,127,131,135,159,215,243,267,271,275,279,363,371,375,379,383,387,391,395,399,403,471,475,479,495,511,515,519,523]]

col_names= ['TotHH','Neigh','Unemploy','<HighSDip','HighSDip','AssDeg','BacDeg+','Foreign%','Child<18','Inc<35k','Inc_35to50','Inc_50to75','Inc75to100','Age65+','Renter','LangNotEng','Age18_24','Age25_34','Age35_44','Age45_54','Male','Race_Black','Race_Native','Race_Asian','Race_Other','Race_Two+','Race_His','Race_White','Race_POC','Poverty','Poverty_At','Poverty_100to150','Poverty_150to200','CarOwn','Commute<10','Commute_10to19','Commute_20-29','Commute_30+']

SPDem.columns= col_names



for i,square in enumerate(SPDem.columns.values): 
    print(str(i) + ' '+ square)

SPDem.iloc[:,[1,6,7,8,13,14,16,17,18,19,20,21,23,25,26,27,29,33]]
#SPDem= SPDem.query('POLICEGRIDNUMBER in [86.0,87.0,88.0,89.0,67.0,68.0,66.0,106.0,107.0,108.0,109.0,110.0] and YEAROFSTOP in [2018,2019]') 


0 TotHH
1 Neigh
2 Unemploy
3 <HighSDip
4 HighSDip
5 AssDeg
6 BacDeg+
7 Foreign%
8 Child<18
9 Inc<35k
10 Inc_35to50
11 Inc_50to75
12 Inc75to100
13 Age65+
14 Renter
15 LangNotEng
16 Age18_24
17 Age25-34
18 Age35_44
19 Age45_54
20 Male
21 Race_Black
22 Race_Native
23 Race_Asian
24 Race_Other
25 Race_Two+
26 Race_His
27 Race_White
28 Race_POC
29 Poverty
30 Poverty_At
31 Poverty_100to150
32 Poverty_150to200
33 CarOwn
34 Commute<10
35 Commute_10to19
36 Commute_20-29
37 Commute_30+


,Neigh,BacDeg+,Foreign%,Child<18,Age65+,Renter,Age18_24,Age25-34,Age35_44,Age45_54,Male,Race_Black,Race_Asian,Race_Two+,Race_His,Race_White,Poverty,CarOwn
43,Summit Hill,0.046647,0.015740,0.018619,0.016998,0.035263,0.011207,0.019430,0.018443,0.017427,0.024770,0.014972,0.012676,0.009408,0.023951,0.035982,0.033832,0.048802
52,Hamline-Midway,0.042874,0.025916,0.032206,0.018212,0.041966,0.024274,0.024015,0.015256,0.019738,0.031973,0.028882,0.033144,0.021190,0.027848,0.046671,0.047604,0.048242
53,Frogtown/Thomas-Dale,0.024246,0.035857,0.036590,0.017979,0.040061,0.015874,0.020365,0.015369,0.013992,0.032660,0.038754,0.034821,0.017385,0.016920,0.021223,0.046262,0.069280
59,Union Park,0.031072,0.014805,0.016709,0.012547,0.024139,0.025139,0.016687,0.011715,0.010721,0.027574,0.020201,0.010190,0.008851,0.015791,0.032326,0.036860,0.039349


In [86]:


TStop = pd.read_csv('Traffic_Stop_Dataset.csv')
TStop.columns = TStop.columns.str.replace(' ', '')
#print(TStop.head())

#TStop.query('POLICEGRIDNUMBER IN [86

#Get Frogtown Grid Number
FG= TStop.query('POLICEGRIDNUMBER in [86.0,87.0,88.0,89.0,67.0,68.0,66.0,106.0,107.0,108.0,109.0,110.0] and YEAROFSTOP in [2018,2019]') 
#TStop.query('POLICEGRIDNUMBER IN [86,87,88,89,67,68,66,106,107,108,109,110]')
#TStop.query('86< POLICEGRIDNUMBER <91 OR 65< POLICE GRID NUMBER <69 OR 105< POLICE GRID NUMBER <111')

#grades_df.query("Test_3 in [98, 99, 100]")
#TStop.dtypes




KeyError: 'Raceofdriver'

In [92]:
#Find unique values of data set and set of values
FG['RACEOFDRIVER'].unique()
FG['RACEOFDRIVER'].value_counts().to_frame()
FG['YEAROFSTOP'].value_counts().to_frame()
FG['GENDEROFDRIVER'].value_counts().to_frame()
FG['GENDEROFDRIVER'].value_counts().to_frame()
print(FG['RACEOFDRIVER'].value_counts().to_frame())
print(FG['YEAROFSTOP'].value_counts().to_frame())
print(FG['GENDEROFDRIVER'].value_counts().to_frame()) #2000 are missing
print(FG['DRIVERSEARCHED?'].value_counts().to_frame())
print(FG['VEHICLESEARCHED?'].value_counts().to_frame())
print(FG['CITATIONISSUED?'].value_counts().to_frame())
print(FG['AGEOFDRIVER'].value_counts().to_frame())
print(FG['REASONFORSTOP'].value_counts().to_frame())
print(FG['POLICEGRIDNUMBER'].value_counts().to_frame())
print(FG['COUNT'].value_counts().to_frame())


FG.isnull().sum()

                 RACEOFDRIVER
Black                    1551
White                     858
Asian                     252
Other                     184
Latino                    113
Native American            10
      YEAROFSTOP
2018        2968
        GENDEROFDRIVER
Male              1942
Female            1026
     DRIVERSEARCHED?
No              2617
Yes              351
     VEHICLESEARCHED?
No               2661
Yes               307
     CITATIONISSUED?
No              1886
Yes             1082
      AGEOFDRIVER
24.0           42
23.0           41
30.0           41
25.0           40
27.0           37
29.0           35
28.0           33
26.0           30
20.0           30
31.0           30
22.0           29
35.0           28
19.0           27
38.0           27
21.0           24
34.0           23
41.0           22
33.0           22
18.0           21
39.0           21
49.0           21
43.0           19
36.0           19
37.0           18
32.0           17
53.0           16
47.0     

YEAROFSTOP                       0
DATEOFSTOP                       0
RACEOFDRIVER                     0
GENDEROFDRIVER                   0
DRIVERSEARCHED?                  0
VEHICLESEARCHED?                 0
CITATIONISSUED?                  0
AGEOFDRIVER                   1999
REASONFORSTOP                    0
POLICEGRIDNUMBER                 0
LOCATIONOFSTOPBYPOLICEGRID       0
COUNT                            0
dtype: int64